In [37]:
import os
import json
import pandas as pd
import re

In [4]:
folder_path = ''  # 文件夹路径
excel_file = '/overall' 

In [5]:
data = pd.DataFrame(columns=['id', 'text', 'conversation_id', 'EXP INT', 'INT'])

In [6]:
for i in range(10):
    folder_name = f'Project_{i}'
    folder_dir = os.path.join(folder_path, folder_name)

    for file_name in os.listdir(folder_dir):
        if file_name.endswith('.json'):
            json_file = os.path.join(folder_dir, file_name)

            with open(json_file, 'r', encoding='utf-8') as f:
                json_data = json.load(f)

            for item in json_data:
                item_data = {
                    'id': item.get('id', ''),
                    'text': item.get('text', ''),
                    'conversation_id': item.get('coversation_id', ''),
                    'EXP INT': item.get('label', [])[0] if item.get('label') else '',
                    'INT': item.get('label', [])[1] if item.get('label') else ''
                }
                data = data.append(item_data, ignore_index=True)


In [7]:
print(data.head)
data.to_csv('overall/overall0.csv', index=False)

<bound method NDFrame.head of          id                                               text  \
0     22313  TEACHER:\n    Hello <STUDENT>!\n==============...   
1     22314  TEACHER:\n    Hi - welcome to the chat\n    Ho...   
2     22315  TEACHER:\n    Oh Austria, interesting!\n    Al...   
3     22316  TEACHER:\n    I'm from Ukraine, but I've lived...   
4     22317  TEACHER:\n    Yes, I was an English teacher an...   
...     ...                                                ...   
8841  47266  TEACHER:\n    I suppose there aren't many new ...   
8842  47267  TEACHER:\n    I haven't seen this one, but I t...   
8843  47268  TEACHER:\n    Ok, I'll let you go! Have a love...   
8844  47269  TEACHER:\n    Bye :)\n========Rate if this tea...   
8845  47270  ===================This is the end of your tas...   

     conversation_id    EXP INT    INT  
0                118  EXP INT 0  INT 0  
1                118  EXP INT 2  INT 2  
2                118  EXP INT 1  INT 1  
3            

In [33]:
# 读取CSV文件
df = pd.read_csv('overall/overall0.csv')

# 提取'EXP INT'列中的数字
df['EXP INT'] = df['EXP INT'].str.extract(r'(\d+)', expand=False).astype(float)

# 提取'INT'列中的数字
df['INT'] = df['INT'].str.extract(r'(\d+)', expand=False).astype(float)

# 显示修改后的DataFrame
print(df)

# 保存修改后的DataFrame为新的CSV文件
df.to_csv('overall/overall.csv', index=False)

         id                                               text  \
0     22313  TEACHER:\n    Hello <STUDENT>!\n==============...   
1     22314  TEACHER:\n    Hi - welcome to the chat\n    Ho...   
2     22315  TEACHER:\n    Oh Austria, interesting!\n    Al...   
3     22316  TEACHER:\n    I'm from Ukraine, but I've lived...   
4     22317  TEACHER:\n    Yes, I was an English teacher an...   
...     ...                                                ...   
8841  47266  TEACHER:\n    I suppose there aren't many new ...   
8842  47267  TEACHER:\n    I haven't seen this one, but I t...   
8843  47268  TEACHER:\n    Ok, I'll let you go! Have a love...   
8844  47269  TEACHER:\n    Bye :)\n========Rate if this tea...   
8845  47270  ===================This is the end of your tas...   

     conversation_id  EXP INT  INT  
0                118      0.0  0.0  
1                118      2.0  2.0  
2                118      1.0  1.0  
3                118      2.0  1.0  
4                118  

In [36]:
# 对"text"列进行预处理
df['text'] = df['text'].apply(lambda x: re.sub(r'=Rate if this teacher finds the student interesting \(please don\'t use your own preferences\)=', '', x))
df['text'] = df['text'].apply(lambda x: re.sub(r'<TEACHER>|<STUDENT>|=|Separation Line', '', x))
df['text'] = df['text'].str.strip()

# 显示修改后的DataFrame
print(df)

# 保存修改后的DataFrame为新的CSV文件
df.to_csv('overall/overall_remove.csv', index=False)

         id                                               text  \
0     22313        TEACHER:\n    Hello !\n\nSTUDENT:\n    Hi !   
1     22314  TEACHER:\n    Hi - welcome to the chat\n    Ho...   
2     22315  TEACHER:\n    Oh Austria, interesting!\n    Al...   
3     22316  TEACHER:\n    I'm from Ukraine, but I've lived...   
4     22317  TEACHER:\n    Yes, I was an English teacher an...   
...     ...                                                ...   
8841  47266  TEACHER:\n    I suppose there aren't many new ...   
8842  47267  TEACHER:\n    I haven't seen this one, but I t...   
8843  47268  TEACHER:\n    Ok, I'll let you go! Have a love...   
8844  47269        TEACHER:\n    Bye :)\n\nSTUDENT:\n    bye )   
8845  47270  This is the end of your task, congrates!\nPlea...   

     conversation_id  EXP INT  INT  
0                118      0.0  0.0  
1                118      2.0  2.0  
2                118      1.0  1.0  
3                118      2.0  1.0  
4                118  

In [38]:
df = pd.read_csv('overall/overall_remove.csv')

# 创建目标文件夹
output_folder = os.path.join('overall', 'txt')
os.makedirs(output_folder, exist_ok=True)

# 遍历DataFrame的每一行
for index, row in df.iterrows():
    conversation_id = row['conversation_id']
    id = row['id']
    text = row['text']

    # 构建txt文件名
    txt_filename = f"{conversation_id}_{id}.txt"

    # 检查是否已存在相同的txt文件，如果存在则跳过该条数据
    if os.path.exists(os.path.join(output_folder, txt_filename)):
        continue

    # 保存文本内容到txt文件
    txt_filepath = os.path.join(output_folder, txt_filename)
    with open(txt_filepath, 'w', encoding='utf-8') as f:
        f.write(text)

In [102]:
import pandas as pd

# 读取数据集
df = pd.read_csv('overall/overall_remove.csv', delimiter=',')

# 更新最后一组相同conversation_id的行
if merged_texts:
    merged_id = merged_ids[0]
    df.loc[len(df)-len(merged_texts):, 'text'] = merged_texts
    df.loc[len(df)-len(merged_texts):, 'id'] = merged_id


# 初始化变量
merged_text = []
merged_id = None
prev_exp_int = None
prev_int = None
prev_conversation_id = None

# 遍历数据集
for index, row in df.iterrows():
    current_exp_int = row['EXP INT']
    current_int = row['INT']
    current_conversation_id = row['conversation_id']
    current_text = row['text']

    if (
        current_exp_int == prev_exp_int and
        current_int == prev_int and
        current_conversation_id == prev_conversation_id
    ):
        merged_text.append(current_text)
        merged_ids.append(row['id'])

    else:
        if merged_id is not None:
            merged_row = [merged_id, ' '.join(merged_text), prev_conversation_id, prev_exp_int, prev_int]
            df.loc[index-1] = merged_row
        merged_text = [current_text]
        # merged_id = row['id']
        
        if merged_texts:
            merged_id = merged_ids[0]
            df.loc[index-len(merged_texts):index-1, 'id'] = merged_id
        merged_ids = [row['id']]
    
    prev_exp_int = current_exp_int
    prev_int = current_int
    prev_conversation_id = current_conversation_id

if merged_texts:
    merged_id = merged_ids[0]
    df.loc[len(df)-len(merged_texts):, 'text'] = merged_texts
    df.loc[len(df)-len(merged_texts):, 'id'] = merged_id
    
# 更新最后一组相同conversation_id的行
# if merged_id is not None:
#     merged_row = [merged_id, ' '.join(merged_text), prev_conversation_id, prev_exp_int, prev_int]
#     df.iloc[-1] = merged_row

# 删除重复的行
# df = df.drop_duplicates(subset='conversation_id')

# 重新设置索引
df = df.reset_index(drop=True)

# 保存修改后的DataFrame为新的CSV文件
df.to_csv('overall/overall_remove_combine.csv', index=False)

In [104]:
df = pd.read_csv('overall/overall_remove_combine.csv')

# 创建目标文件夹
output_folder = os.path.join('overall', 'txt')
os.makedirs(output_folder, exist_ok=True)

# 遍历DataFrame的每一行
for index, row in df.iterrows():
    conversation_id = row['conversation_id']
    id = row['id']
    text = row['text']

    # 构建txt文件名
    txt_filename = f"{conversation_id}_{id}.txt"

    # 检查是否已存在相同的txt文件
    if os.path.exists(os.path.join(output_folder, txt_filename)):
        # 获取已存在文件的路径
        existing_txt_filepath = os.path.join(output_folder, txt_filename)

        # 读取已存在文件的内容
        with open(existing_txt_filepath, 'r', encoding='utf-8') as f:
            existing_text = f.read()

        # 比较文本字数，保留字数最多的文本
        if len(existing_text) > len(text):
            continue

    # 保存文本内容到txt文件
    txt_filepath = os.path.join(output_folder, txt_filename)
    with open(txt_filepath, 'w', encoding='utf-8') as f:
        f.write(text)

In [35]:
import pandas as pd
import re

# 读取数据集
df = pd.read_csv('overall/overall_remove_combine.csv')

# 将换行符转换为空格
df['text'] = df['text'].replace('\n', ' ', regex=True)

# 删除学生说的话
df['text'] = df['text'].apply(lambda x: re.sub(r'STUDENT:.*?(?=TEACHER:|$)', '', x))

# 去除多余的空格
df['text'] = df['text'].str.strip()

# 显示修改后的DataFrame
print(df)

# 保存修改后的DataFrame为新的CSV文件
df.to_csv('overall/overall_remove_student.csv', index=False)

         id                                               text  \
0     22313                               TEACHER:     Hello !   
1     22314  TEACHER:     Hi - welcome to the chat     How ...   
2     22315  TEACHER:     Oh Austria, interesting!     All ...   
3     22316  TEACHER:     I'm from Ukraine, but I've lived ...   
4     22317  TEACHER:     Yes, I was an English teacher and...   
...     ...                                                ...   
8841  47266  TEACHER:     I suppose there aren't many new f...   
8842  47267  TEACHER:     I haven't seen this one, but I th...   
8843  47268  TEACHER:     Ok, I'll let you go! Have a lovel...   
8844  47269                                TEACHER:     Bye :)   
8845  47270  This is the end of your task, congrates! Pleas...   

     conversation_id  EXP INT  INT  
0                118      0.0  0.0  
1                118      2.0  2.0  
2                118      1.0  1.0  
3                118      2.0  1.0  
4                118  

In [38]:
df = pd.read_csv('overall/overall_remove_student.csv')

# 创建目标文件夹
output_folder = os.path.join('overall', 'txt')
os.makedirs(output_folder, exist_ok=True)

# 遍历DataFrame的每一行
for index, row in df.iterrows():
    conversation_id = row['conversation_id']
    id = row['id']
    text = row['text']

    # 构建txt文件名
    txt_filename = f"{conversation_id}_{id}.txt"

    # 检查是否已存在相同的txt文件
    if os.path.exists(os.path.join(output_folder, txt_filename)):
        # 获取已存在文件的路径
        existing_txt_filepath = os.path.join(output_folder, txt_filename)

        # 读取已存在文件的内容
        with open(existing_txt_filepath, 'r', encoding='utf-8') as f:
            existing_text = f.read()

        # 比较文本字数，保留字数最多的文本
        if len(existing_text) > len(text):
            continue

    # 保存文本内容到txt文件
    txt_filepath = os.path.join(output_folder, txt_filename)
    with open(txt_filepath, 'w', encoding='utf-8') as f:
        f.write(text)

In [23]:
import pandas as pd

# 读取数据集
df = pd.read_csv('overall/overall_remove.csv', delimiter=',')

# 初始化变量
merged_texts = []
merged_ids = None
prev_exp_int = None
prev_int = None
prev_conversation_id = None

# 遍历数据集
for index, row in df.iterrows():
    current_exp_int = row['EXP INT']
    current_int = row['INT']
    current_conversation_id = row['conversation_id']
    current_text = row['text']

    if (
        current_exp_int == prev_exp_int and
        current_int == prev_int and
        current_conversation_id == prev_conversation_id
    ):
        merged_texts.append(current_text)
        merged_ids.append(row['id'])
    else:
        if merged_texts:
            merged_id = merged_ids[0]
            df.loc[index-len(merged_texts):index-1, 'text'] = merged_texts
            df.loc[index-len(merged_texts):index-1, 'id'] = merged_id
        merged_texts = [current_text]
        merged_ids = [row['id']]
    
    prev_exp_int = current_exp_int
    prev_int = current_int
    prev_conversation_id = current_conversation_id

# 更新最后一组相同conversation_id的行
if merged_texts:
    merged_id = merged_ids[0]
    df.loc[len(df)-len(merged_texts):, 'text'] = merged_texts
    df.loc[len(df)-len(merged_texts):, 'id'] = merged_id

# 删除重复的行
# df = df.drop_duplicates(subset='conversation_id', keep='last')

# 重新设置索引
df = df.reset_index(drop=True)

# 输出结果
print(df.head(20))

       id                                               text conversation_id  \
0   22313        TEACHER:\n    Hello !\n\nSTUDENT:\n    Hi !             118   
1   22314  TEACHER:\n    Hi - welcome to the chat\n    Ho...             118   
2   22315  TEACHER:\n    Oh Austria, interesting!\n    Al...             118   
3   22316  TEACHER:\n    I'm from Ukraine, but I've lived...             118   
4   22317  TEACHER:\n    Yes, I was an English teacher an...             118   
5   22318  TEACHER:\n    I don't really count German amon...             118   
6   22319  TEACHER:\n    Do nouns have genders in Italian...             118   
7   22320  TEACHER:\n    Yes, very similar to Russian / U...             118   
8   22321  TEACHER:\n    Haha, I see your point!!\n    Bu...             118   
9   22322  TEACHER:\n    Hmm, that's an interesting obser...             118   
10  22323  TEACHER:\n    Interesting, I've got to think a...             118   
11  22324  TEACHER:\n    Yes, that's it,

In [1]:
import pandas as pd

# 读取数据
data = {
    'id': [22313],
    'text': ["TEACHER:\nHello <STUDENT>!\n===========================Separation Line=============================\nSTUDENT:\nHi <TEACHER>!"],
    'conversation_id': [118],
    'EXP INT': [0],
    'INT': [0]
}
df = pd.DataFrame(data)

# 删除"===========================Separation Line============================="及其之前的内容
df['text'] = df['text'].apply(lambda x: x.split("===========================Separation Line=============================")[-1].strip())

# 打印处理后的结果
print(df)


      id                     text  conversation_id  EXP INT  INT
0  22313  STUDENT:\nHi <TEACHER>!              118        0    0


In [3]:
import re

# 读取CSV文件
df = pd.read_csv('overall/overall.csv')

# 对"text"列进行预处理
df['text'] = df['text'].apply(lambda x: x.split("===========================Separation Line=============================")[-1].strip())
df['text'] = df['text'].apply(lambda x: re.sub(r'=Rate if this teacher finds the student interesting \(please don\'t use your own preferences\)=', '', x))
df['text'] = df['text'].apply(lambda x: re.sub(r'<TEACHER>|<STUDENT>|=|Separation Line', '', x))
df['text'] = df['text'].str.strip()

# 显示修改后的DataFrame
print(df)

# 保存修改后的DataFrame为新的CSV文件
df.to_csv('overall/overall_student_teacher.csv', index=False)

         id                                               text  \
0     22313                                 STUDENT:\n    Hi !   
1     22314  STUDENT:\n    Thank you! And thank you for let...   
2     22315  STUDENT:\n    Yes, it is a nice place to be :)...   
3     22316  STUDENT:\n    Oh, I see...so you also know wha...   
4     22317  STUDENT:\n    Sort of polyglot...but I enjoy l...   
...     ...                                                ...   
8841  47266  TEACHER:\n    I suppose there aren't many new ...   
8842  47267  TEACHER:\n    I haven't seen this one, but I t...   
8843  47268  TEACHER:\n    Ok, I'll let you go! Have a love...   
8844  47269        TEACHER:\n    Bye :)\n\nSTUDENT:\n    bye )   
8845  47270  This is the end of your task, congrates!\nPlea...   

     conversation_id  EXP INT  INT  
0                118      0.0  0.0  
1                118      2.0  2.0  
2                118      1.0  1.0  
3                118      2.0  1.0  
4                118  